In [121]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras.backend as K
from tensorflow import keras
from keras.layers import Reshape, Conv2DTranspose, PReLU
from keras.utils import to_categorical, plot_model
from keras.layers import Concatenate
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.image import resize
from scipy.linalg import sqrtm
import tensorflow as tf
from IPython.display import clear_output, HTML
import glob
from keras.layers import AveragePooling2D, ZeroPadding2D, BatchNormalization, Activation, MaxPool2D, Add
from keras.layers import Normalization, Dense, Conv2D, Dropout, BatchNormalization, ReLU
from keras.models import Sequential, Model
from keras import Input
from keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU, GlobalMaxPooling2D, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import numpy as np
from skimage.transform import AffineTransform, warp
from keras import Sequential, Model
from keras.layers import Dense, Conv2D, Reshape, UpSampling2D, \
    BatchNormalization, Activation, Input, LeakyReLU, ZeroPadding2D, Dropout, Flatten, Conv2DTranspose
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import rotate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation, Input, LeakyReLU
from keras.initializers import RandomNormal
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout

In [122]:
sns.set(style='darkgrid', palette='deep')

In [123]:
# List physical GPUs and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [124]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('emnist-letters-train.csv', delimiter=',', header=None)

In [125]:
df.rename(columns={0: 'labels'}, inplace=True)

In [126]:
df = df[df['labels'] != -1]


In [127]:
df

,labels,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99032,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99033,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99035,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99036,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
# Define the mapping dictionary
mapping = {1: 97, 2: 98, 3: 99, 4: 100, 5: 101, 6: 102, 7: 103, 8: 104, 9: 105, 
           10: 106, 11: 107, 12: 108, 13: 109, 14: 110, 15: 111, 16: 112, 17: 113, 
           18: 114, 19: 115, 20: 116, 21: 117, 22: 118, 23: 119, 24: 120, 25: 121, 
           26: 122, 27: 123}

        # Map the labels column to its corresponding value
df['labels'] = df['labels'].map(mapping)

In [129]:
# Convert the DataFrame to a NumPy array
data = df.to_numpy()

# Extract the labels and images from the NumPy array
labels = data[:, 0]  # Assuming the first column is the labels
images = data[:, 1:]  # The rest are the images

# Optionally, you can reshape and normalize the images if needed
# For example, if the images are 28x28 pixels
images = images.reshape(-1, 28, 28, 1)  # Reshape to (num_samples, 28, 28, 1)
# images = images / 255.0  # Normalize to [0, 1]

# Convert the mapped labels to characters
# mapped_labels = np.vectorize(mapping.get)(labels)
# labels = np.vectorize(chr)(mapped_labels)

# Now 'images' and 'labels' are NumPy arrays ready for use
print(images.shape)
print(labels.shape)

(88800, 28, 28, 1)
(88800,)


In [130]:
labels

array([119, 103, 112, ..., 114, 120, 115], dtype=int64)

In [131]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

Training set shape: (56832, 28, 28, 1)
Validation set shape: (14208, 28, 28, 1)
Test set shape: (17760, 28, 28, 1)


In [132]:
unique_labels = np.unique(y_train)
def Show_Images(data,unique_labels,labels):
    # Create the plot with sufficient subplots
    fig, axes = plt.subplots(3, 9, figsize=(20, 8))  # Adjusted to 3 rows and 9 columns for demonstration
    axes = axes.flatten()

    for label, ax in zip(unique_labels, axes):
        # Select the first image for each unique label
        idx = np.where(labels == label)[0][0]
        image = data[idx].reshape(28, 28)
        
        # Display the image
        ax.imshow(image, cmap='gray')
        ax.set_title(f'Label: {label}')
        ax.axis('off')

    # Hide any remaining empty subplots (if any)
    for ax in axes[len(unique_labels):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Show_Images(X_train,unique_labels,y_train)

In [133]:
def plot_images_per_label_Aug_sheer(images, labels, num_images_per_label=5):
    unique_labels = np.unique(labels)
    
    # Create the plot with sufficient subplots
    fig, axes = plt.subplots(len(unique_labels), num_images_per_label * 2, figsize=(20, len(unique_labels) * 2))
    axes = axes.flatten()  # Flatten the 2D array of axes to easily iterate

    # Counter for current axis
    ax_index = 0

    for label in unique_labels:
        # Filter images for the current label
        indices = np.where(labels == label)[0]
        
        # Loop through up to num_images_per_label images for the current label
        for i in range(min(num_images_per_label, len(indices))):
            image = images[indices[i]].reshape(28, 28)
            
            # Display the original image
            axes[ax_index].imshow(image, cmap='gray')
            axes[ax_index].set_title(f'Label: {label}')
            axes[ax_index].axis('off')
            ax_index += 1

            if ax_index >= len(axes):
                break

        for i in range(min(num_images_per_label, len(indices))):
            image = images[indices[i]].reshape(28, 28)

            # Rotate the image by 15 degrees
            rotated_image = rotate(image, -90, reshape=False)
            
            # Apply shear transformation
            # shear_transform = AffineTransform(shear=0.2)
            # sheared_image = warp(rotated_image, shear_transform.inverse, mode='wrap')
            flipped_image = np.fliplr(rotated_image)

            # Display the augmented image
            axes[ax_index].imshow(flipped_image, cmap='gray')
            axes[ax_index].set_title(f'Labels Augmented: {label}')
            axes[ax_index].axis('off')
            ax_index += 1

            if ax_index >= len(axes):
                break

    # Hide any remaining empty subplots (if any)
    for ax in axes[ax_index:]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [134]:
# plot_images_per_label_Aug_sheer(X_train, y_train)

In [135]:
augmented_images = []
for image in images:
    rotated_image = rotate(image, 90, reshape=False)
    flipped_image = np.flipud(rotated_image)  # Flip vertically
    augmented_images.append(flipped_image)

In [136]:
augmented_images = np.array(augmented_images)

In [137]:
# Split the data into train and test sets
X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(augmented_images, labels, test_size=0.05, random_state=42)
print(f"Training Augmented set shape: {X_train_aug.shape}")
print(f"Test Augmented set shape: {X_test.shape}")

Training Augmented set shape: (84360, 28, 28, 1)
Test Augmented set shape: (17760, 28, 28, 1)


In [138]:
print(f'Augmented Shape : {augmented_images.shape}')
print(f'Label Shape : {labels.shape}')

Augmented Shape : (88800, 28, 28, 1)
Label Shape : (88800,)


In [139]:
# class DCGAN():
#     def __init__(self, rows, cols, channels, z = 100):
#         # Input shape
#         self.img_rows = rows
#         self.img_cols = cols
#         self.channels = channels
#         self.img_shape = (self.img_rows, self.img_cols, self.channels)
#         self.latent_dim = z
#         optimizer = Adam(0.0001, 0.7)
#         self.discriminator = self.build_discriminator()
#         self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
#         self.generator = self.build_generator()
#         z = Input(shape=(self.latent_dim,))
#         img = self.generator(z)
#         self.discriminator.trainable = False
#         valid = self.discriminator(img)
#         self.combined = Model(z, valid)
#         self.combined.compile(loss='binary_crossentropy',optimizer=optimizer)
    
#     def train(self, epochs, batch_size=256, save_interval=50):
#         # Load the dataset
#         X_train = X_train_aug
#         # Rescale -1 to 1
#         X_train = X_train / 255
#         X_train = np.expand_dims(X_train, axis=3)
#         # Adversarial ground truths
#         valid = np.ones((batch_size, 1))
#         fake = np.zeros((batch_size, 1))
#         for epoch in range(epochs):
#             idx = np.random.randint(0, X_train.shape[0], batch_size)
#             imgs = X_train[idx]
#             # Sample noise and generate a batch of new images
#             noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
#             gen_imgs = self.generator.predict(noise)
#             # Train the discriminator (real classified as ones
#             # and generated as zeros)
#             d_loss_real = self.discriminator.train_on_batch(imgs, valid)
#             d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
#             d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
#             # ---------------------
#             # Train Generator
#             # ---------------------
#             # Train the generator (wants discriminator to mistake
#             # images as real)
#             g_loss = self.combined.train_on_batch(noise, valid)
#             # Plot the progress
#             print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

#             # If at save interval => save generated image samples
#             if epoch % save_interval == 0:
#                 self.save_imgs(epoch)

#     def build_discriminator(self):
#         model = Sequential()
#         model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
#         model.add(ZeroPadding2D(padding=((0,1),(0,1))))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(528, kernel_size=3, strides=1, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(1024, kernel_size=3, strides=1, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Flatten())
#         model.add(Dense(1, activation='sigmoid'))
#         model.summary()
#         img = Input(shape=self.img_shape)
#         validity = model(img)
#         return Model(img, validity)
    
#     def build_generator(self):
#         model = Sequential()
#         model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
#         model.add(Reshape((7, 7, 128)))
#         model.add(UpSampling2D())
#         model.add(Conv2D(128, kernel_size=3, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(Activation("relu"))
#         model.add(UpSampling2D())
#         model.add(Conv2D(128, kernel_size=3, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(Activation("relu"))
#         model.add(Conv2D(64, kernel_size=3, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
#         model.add(Activation("relu"))
#         model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
#         model.add(Activation("tanh"))
#         model.summary()
#         noise = Input(shape=(self.latent_dim,))
#         img = model(noise)
#         return Model(noise, img)
    
#     def save_imgs(self, epoch):
#         r, c = 5, 5
#         noise = np.random.normal(0, 1, (r * c, self.latent_dim))
#         gen_imgs = self.generator.predict(noise)

#         # Rescale images 0 - 1
#         # gen_imgs = 0.5 * gen_imgs + 0.5

#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             for j in range(c):
#                 axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         os.makedirs('generated_mnist', exist_ok=True)
#         fig.savefig("generated_mnist/dcgan_mnist_improved_{:d}.png".format(epoch))
#         plt.close()

In [140]:
# # Set image dimensions
# img_rows, img_cols, channels = 28, 28, 1

# # Instantiate and train the DCGAN
# dcgan = DCGAN(img_rows, img_cols, channels)
# dcgan.train(epochs=10000, batch_size=256, save_interval=50)

In [141]:
# class DCGAN():

#     # this is the function to build the generator neural network
#     def build_generator(self):
#         model = Sequential(name='Generator')
#         model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim)) # connect the input to dense layer
#         model.add(Reshape((7, 7, 128)))
#         # upsample from 7*7 to 14*14
#         model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#         model.add(LeakyReLU(alpha=0.2))
#         # upsample to 28x28
#         model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Conv2D(self.channels, kernel_size=7, padding="same", activation='sigmoid'))
#         model.summary()
#         noise = Input(shape=(self.latent_dim,))
#         img = model(noise)
#         return Model(noise, img)  # the keras Model class groups layers into an object with training and inference features
    
#     def build_discriminator(self):
#         model = Sequential(name='Discriminator')
#         model.add(Conv2D(128, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.4))
#         model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.4))
#         model.add(Flatten())
#         model.add(Dense(1, activation='sigmoid'))
#         model.summary()
#         img = Input(shape=self.img_shape)
#         validity = model(img)
#         return Model(img, validity)
    
#     def __init__(self, rows, cols, channels, z = 100):
#         # Input shape
#         self.img_rows = rows  # generated image height
#         self.img_cols = cols  # generated image width
#         self.channels = channels  # generated image channel
#         self.img_shape = (self.img_rows, self.img_cols, self.channels)
#         self.latent_dim = z  # the input is 1-D vector of noise
#         # Reduce learning rate from 0.001 to 0.0002, and beta1 from 0.9 to 0.5, which can stablize training and reduce oscillation
#         optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
#         # Build and compile the discriminator
#         self.discriminator = self.build_discriminator()
#         self.discriminator.compile(loss='binary_crossentropy',
#             optimizer=optimizer,
#             metrics=['accuracy'])
#         # Build the generator
#         self.generator = self.build_generator()
#         # The generator takes noise as input and generates images
#         z = Input(shape=(self.latent_dim,))
#         img = self.generator(z)
#         # For the combined model we will only train the generator
#         self.discriminator.trainable = False
#         # The discriminator takes generated images as input and determines validity
#         valid = self.discriminator(img)
#         # The combined model (stacked generator and discriminator)
#         # Trains the generator to fool the discriminator
#         self.combined = Model(z, valid)
#         self.combined.compile(loss='binary_crossentropy',
#             optimizer=optimizer)
    
#     def train(self, epochs, batch_size=128, save_interval=50):
#         # Load the dataset
#         # Split the data into train and test sets
#         X_train, X_test, y_train, y_test = train_test_split(augmented_images, labels, test_size=0.2, random_state=42)
#         # Rescale 0 to 1
#         X_train = X_train / 255
#         X_train = np.expand_dims(X_train, axis=3)
#         # Adversarial ground truths
#         valid = np.ones((batch_size, 1))
#         fake = np.zeros((batch_size, 1))
#         for epoch in range(epochs):
#             # ---------------------
#             # Train Discriminator
#             # ---------------------
#             # Select a random half of images
#             idx = np.random.randint(0, X_train.shape[0], batch_size)
#             imgs = X_train[idx]
#             # Sample noise and generate a batch of new images
#             noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
#             gen_imgs = self.generator.predict(noise)
#             # Train the discriminator (it classify real images as 1 and generated images as 0)
#             d_loss_real = self.discriminator.train_on_batch(imgs, valid)
#             d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
#             d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
#             # Train the generator (it wants discriminator to predict generated images as 1)
#             g_loss = self.combined.train_on_batch(noise, valid)
#             # Plot the progress
#             print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
#             # If at save interval => save generated image samples
#             if epoch % save_interval == 0:
#                 self.save_imgs(epoch)
    
#     def save_imgs(self, epoch):
#         r, c = 5, 5
#         noise = np.random.normal(0, 1, (r * c, self.latent_dim))
#         gen_imgs = self.generator.predict(noise)

#         # Rescale images 0 - 1
#         # gen_imgs = 0.5 * gen_imgs + 0.5

#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             for j in range(c):
#                 axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#                 axs[i,j].axis('off')
#                 cnt += 1
#         os.makedirs('generated_mnist', exist_ok=True)
#         fig.savefig("generated_mnist/dcgan_mnist_{:d}.png".format(epoch))
#         plt.close()

In [142]:
# # Set image dimensions
# img_rows, img_cols, channels = 28, 28, 1

# # Instantiate and train the DCGAN
# dcgan = DCGAN(img_rows, img_cols, channels)
# dcgan.train(epochs=10000, batch_size=256, save_interval=50)

In [147]:
def build_conditional_generator(z_dim=100, output_shape=(88800, 3, 1),
                                num_classes=3, verbose=True):
    noise = Input(shape=(z_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, z_dim)(label))
    model_input = multiply([noise, label_embedding])

    mlp = Dense(256, input_dim=z_dim)(model_input)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = BatchNormalization(momentum=0.8)(mlp)
    mlp = Dense(512)(mlp)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = Dense(1024)(mlp)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = BatchNormalization(momentum=0.8)(mlp)
    mlp = Dense(np.prod(output_shape), activation='tanh')(mlp)
    mlp = Reshape(output_shape)(mlp)

    model = Model([noise, label], mlp)

    if verbose:
        model.summary()
    return model


def build_conditional_discriminator(input_shape=(1200, 3, 1),
                                    num_classes=3, verbose=True):
    img = Input(shape=input_shape)
    flat_img = Flatten()(img)

    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes,
                                          np.prod(input_shape))(label))

    model_input = multiply([flat_img, label_embedding])

    mlp = Dense(512, input_dim=np.prod(input_shape))(model_input)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = Dense(512)(mlp)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = Dropout(0.4)(mlp)
    mlp = Dense(512)(mlp)
    mlp = LeakyReLU(alpha=0.2)(mlp)
    mlp = Dropout(0.4)(mlp)
    mlp = Dense(1, activation='sigmoid')(mlp)

    model = Model([img, label], mlp)

    if verbose:
        model.summary()

    return model


def build_critic(input_shape=(1200, 3, 1), verbose=True):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(16, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))

    if verbose:
        model.summary()
    return model


def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)



def train(generator=None,discriminator=None,gan_model=None,
          epochs=1000, batch_size=128, sample_interval=50,
          z_dim=100):
    labels = data[:, 0]  # Assuming the first column is the labels
    # Load MNIST train samples
    X_train, X_test_aug, y_train, y_test_aug = train_test_split(augmented_images, labels, test_size=0.2, random_state=42)

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Prepare GAN output labels
    real_y = np.ones((batch_size, 1))
    fake_y = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # train disriminator
        # pick random real samples from X_train
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs, labels = X_train[idx], y_train[idx]

        # pick random noise samples (z) from a normal distribution
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        # use generator model to generate output samples
        fake_imgs = generator.predict([noise, labels])

        # calculate discriminator loss on real samples
        disc_loss_real = discriminator.train_on_batch([real_imgs, labels], real_y)
        
        # calculate discriminator loss on fake samples
        disc_loss_fake = discriminator.train_on_batch([fake_imgs, labels], fake_y)
        
        # overall discriminator loss
        discriminator_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake)
        
        # train generator
        # pick random noise samples (z) from a normal distribution
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        
        # pick random labels for conditioning
        sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

        # use trained discriminator to improve generator
        gen_loss = gan_model.train_on_batch([noise, sampled_labels], real_y)

        # training updates
        print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, 
                                                                                   discriminator_loss[0], 
                                                                                   100*discriminator_loss[1], 
                                                                                   gen_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch,generator)


def sample_images(epoch, generator, z_dim=100,
                  save_output=True,
                  output_dir="images"):

    # get label if conditional generator is active
    model_type = 'cgan' if isinstance(generator.input_shape, list) else 'others'

    if model_type == 'cgan':
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, z_dim))
        sampled_labels = np.arange(0,2).reshape(-1, 1)
        gen_imgs = generator.predict([noise, sampled_labels])
    else:
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, z_dim))
        gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    # get output shape
    output_shape = len(generator.output_shape)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            if output_shape == 3:
                axs[i, j].imshow(gen_imgs[cnt, :, :], cmap='gray')
            else:
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            if model_type == 'cgan':
                axs[i, j].set_title("Label: %d" % sampled_labels[cnt])
            axs[i, j].axis('off')
            cnt += 1
    plt.show()
    if save_output:
        fig.savefig("{}/{}.png".format(output_dir, epoch))
    plt.close()



In [148]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import multiply
#Code snippet for creating and compiling the cGAN model
discriminator = build_conditional_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
generator = build_conditional_generator()

def build_generator(z_dim):
    model = Sequential()

    model.add(Dense(256, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))

    return model

def build_discriminator():
    model = Sequential()

    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

def build_gan(generator, discriminator):
    z = Input(shape=(100,))
    img = generator(z)
    discriminator.trainable = False
    valid = discriminator(img)
    
    combined = Model(z, valid)
    combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    
    return combined

# Create the models
generator = build_generator(100)
discriminator = build_discriminator()

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Create and compile the GAN model
gan_model = build_gan(generator, discriminator)

# Code snippet for training the cGAN
train(generator, discriminator, gan_model, epochs=20000, batch_size=32, sample_interval=100)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 1200, 3, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 embedding_9 (Embedding)        (None, 1, 3600)      10800       ['input_25[0][0]']               
                                                                                                  
 flatten_19 (Flatten)           (None, 3600)         0           ['input_24[0][0]']        

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]